# <a href="https://www.pythontutorial.net/advanced-python/python-closures/" style="color:Tomato">Python Closures</a>

Trong bài này, ta sẽ học về Python closures và các ứng dụng thực tế của chúng.

### Tables of Contents
* [Introduction to the Python closures](#1)
* [Python cells and multi-scoped variables](#2)
* [When Python creates the closure](#3)
* [Python closures and for loop](#4)
* [Summary](#sum)

## <a class="anchor" id="1">Introduction to the Python closures</a>

Trong Python, ta có thể định nghĩa các hàm ở bên trong một hàm khác. Đây được gọi là _nested function_. Ví dụ:

In [1]:
def say():
    greeting = 'Hello'

    def display():
        print(greeting)

    display()

Trong ví dụ này, ta định nghĩa hàm `display()` bên trong hàm `say()`. Hàm `display()` được gọi là nested function.

Ở trong hàm `display()`, ta truy cập vào biến `greeting` từ nonlocal scope. Python gọi biến `greeting` này là biến free (_free variable_).

Khi nhìn vào hàm `display()`, thực ra là ta nhìn vào:
- Bản thân hàm `display()`
- Biến free `greeting` với giá trị là `'Hello'`

Sự kết hợp giữa hàm `display()` và biến `greeting` trong trường hợp này gọi là một _closure_.
![](https://www.pythontutorial.net/wp-content/uploads/2020/11/Python-Closure-Example.png)

**Định nghĩa:** <span style="color:DarkOrange">Một **closure** là một nested function và tham chiếu tới một hoặc nhiều biến ở scope bao quanh nó</span>.

Trong Python, một function có thể trả về giá trị là một function khác. Ví dụ:

In [2]:
def say():
    greeting = 'Hello'

    def display():
        print(greeting)

    return display    


Trong ví dụ này, hàm `say()` trả về hàm `display` thay vì thực thi nó.

Khi hàm `say()` trả về hàm `display`, thực ra nó trả về closure của hàm `display`.
![](https://www.pythontutorial.net/wp-content/uploads/2020/11/Python-Closures.png)

Phép gán sau thực hiện gán giá trị của hàm `say()` vào biến `fn`. Vì `fn` cũng là một hàm, nên ta có thể thực thi nó:

In [3]:
fn = say()
fn()

Hello


Hàm `say()` thực thi và return một function. Khi hàm `fn()` được thực thi, hàm `say()` thực tế đã được thực thi xong.

Nói cách khác, scope của hàm `say()` đã hoàn toàn biến mất khi hàm `fn()` được thực thi. Vì biến `greeting` thuộc về scope của hàm `say()` nên nó cũng bị biến mất theo.

Tuy nhiên bạn vẫn sẽ thấy `fn` trả về được kết quả.

## <a class="anchor" id="2">Python cells and multi-scoped variables</a>

Giá trị của biến `greeting` được chia sẻ giữa 2 scopes:
- Hàm `say()`
- Closure

Tên biến `greeting` nằm ở 2 scopes khác nhau, tuy nhiên chúng sẽ tham chiếu tới cùng một object với giá trị là `'Hello'`.

Để làm được như vậy, Python tạo ra một object trung gian được gọi là `cell`.
![](https://www.pythontutorial.net/wp-content/uploads/2022/06/Python-Closures.svg)

Để lấy địa chỉ bộ nhớ của object `cell`, ta sử dụng thuộc tính `__closure__` như sau:

In [4]:
print(fn.__closure__)

(<cell at 0x1068ada20: str object at 0x105ca7d30>,)


Thuộc tính `__closure__` trả về một tuple các cells.

Như trong ví dụ được minh hoạ trong ảnh, địa chỉ của cell là `0x0000017184915C40`, và nó tham chiếu đến một string object có địa chỉ `0x0000017186A829B0`.

Nếu ta in ra địa chỉ của string object ở bên trong hàm `say()` và bên trong closure, ta sẽ thấy chúng tham chiếu tới cùng một object trong bộ nhớ:

In [5]:
def say():
    greeting = 'Hello'
    print(hex(id(greeting)))

    def display():
        print(hex(id(greeting)))
        print(greeting)

    return display


fn = say()
fn()


0x105ca7d30
0x105ca7d30
Hello


Khi ta truy cập vào giá trị của biến `greeting`, Python sẽ thực hiện "tham chiếu hai lần" tới giá trị của string object. Điều này giải thích tại sao khi hàm `say()` nằm ngoài scope, ta vẫn có thể tham chiếu được string object được tham chiếu bởi biến `greeting`.

Với cơ chế này, ta có thể hình dung closure là một hàm với scope được mở rộng để chứa thêm các free variables.

Để xem closure đang chứa những variable nào, ta dùng thuộc tính `__code__.co_freevars`. Ví dụ:

In [6]:
def say():

    greeting = 'Hello'

    def display():
        print(greeting)

    return display


fn = say()
print(fn.__code__.co_freevars)

('greeting',)


Ta thấy `__code__.co_freevars` trả về free variable `greeting` của closure `fn`.

## <a class="anchor" id="3">When Python creates the closure</a>

Python tạo một scope mới khi một hàm được thực thi. Nếu hàm đó tạo ra một closure, Python cũng sẽ tạo ra một closure mới.

Ta xem ví dụ sau:

Đầu tiên, định nghĩa một function `multiplier()` trả về một closure:

In [7]:
def multiplier(x):
    def multiply(y):
        return x * y
    return multiply

Hàm `multiplier()` trả về phép nhân của hai tham số. Tuy nhiên nó sử dụng closure.

Tiếp theo, ta gọi hàm `multiplier()` 3 lần:

In [8]:
m1 = multiplier(1)
m2 = multiplier(2)
m3 = multiplier(3)

Việc này tạo ra 3 closures, mỗi closure nhân một số với lần lượt `1`, `2`, `3`.

Giờ ta thực thi các hàm của các closures đấy:

In [9]:
print(m1(10))
print(m2(10))
print(m3(10))

10
20
30


`m1`, `m2`, `m3` là các instance khác nhau của closure.

## <a class="anchor" id="4">Python closures and for loop</a>

Giả sử ta muốn tạo ra cả 3 closures phía trên chỉ với một lần bằng cách sử dụng vòng lặp:

In [10]:
multipliers = []
for x in range(1, 4):
    multipliers.append(lambda y: x * y)

m1, m2, m3 = multipliers

print(m1(10))
print(m2(10))
print(m3(10))


30
30
30


Ở đây ta sử dụng `lambda expression` để tạo ra closure và thêm closure đó vào trong list `multipliers` sau mỗi bước lặp.

Nó không in ra `10`, `20`, `30` như ta mong muốn.

Ở đây, biến `x` lặp từ `1` đến `3`. Sau vòng lặp, `x` là `3`.

Mỗi phần tử của list là closure:
```python
lambda y: x * y
```

Python sẽ sử dụng biến `x` mỗi khi ta gọi closure này. Ở đây `x` là `3` nên khi ta gọi closure, nó cứ sử dụng `x` là `3`.

Đấy là lý do tại sao cả 3 kết quả đều là `30`.

Để khắc phục, ta sẽ chỉ cho Python biết là phải sử dụng biến `x` ở bên trong vòng lặp:

In [11]:
def multiplier(x):
    def multiply(y):
        return x * y
    return multiply


multipliers = []
for x in range(1, 4):
    multipliers.append(multiplier(x))

m1, m2, m3 = multipliers

print(m1(10))
print(m2(10))
print(m3(10))


10
20
30


## <a class="anchor" id="sum" style="color:Violet"> Tổng kết </a>

Closure là một hàm với scope được mở rộng để chứa thêm các free variables.